In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)     # Show all rows
pd.set_option('display.width', None)        # Width of the display in characters
pd.set_option('display.max_colwidth', None) # Show full content of each 

import warnings 
warnings.filterwarnings('ignore')

In [2]:
root = 'KuaiRec 2.0/'

joined_train_data = pd.read_csv(root + 'data_exports/joined_train_data.csv')
joined_val_data = pd.read_csv(root + 'data_exports/joined_val_data.csv')
joined_test_data = pd.read_csv(root + 'data_exports/joined_test_data.csv')

In [3]:
joined_train_data.columns

Index(['user_id', 'video_id', 'time', 'watch_ratio', 'user_active_degree',
       'is_lowactive_period', 'is_live_streamer', 'is_video_author',
       'follow_user_num', 'fans_user_num', 'friend_user_num', 'register_days',
       'author_id', 'video_type', 'video_tag_name', 'video_duration',
       'show_cnt', 'play_cnt', 'play_duration', 'like_cnt', 'comment_cnt',
       'share_cnt', 'follow_cnt', 'collect_cnt', 'manual_cover_text',
       'caption', 'topic_tag', 'first_level_category_name',
       'second_level_category_name', 'third_level_category_name',
       'english_caption', 'english_first_level_category_name',
       'english_second_level_category_name',
       'english_third_level_category_name', 'english_topic_tag'],
      dtype='object')

In [4]:
joined_train_data.head()

,user_id,video_id,time,watch_ratio,user_active_degree,is_lowactive_period,is_live_streamer,is_video_author,follow_user_num,fans_user_num,friend_user_num,register_days,author_id,video_type,video_tag_name,video_duration,show_cnt,play_cnt,play_duration,like_cnt,comment_cnt,share_cnt,follow_cnt,collect_cnt,manual_cover_text,caption,topic_tag,first_level_category_name,second_level_category_name,third_level_category_name,english_caption,english_first_level_category_name,english_second_level_category_name,english_third_level_category_name,english_topic_tag
0,14,148,2020-07-05 05:27:48.378,0.722103,full_active,0,0,1,73,6,1,279,6332,NORMAL,"女青年, 男孩",6066.0,4403123,4602349,57604317617,29772,4513,545,1933,5.0,UNKNOWN,美60岁奶奶与21少年一见钟情，兴奋分享初次体验，称：升华了感情！ @推广小助手(O40300047),[],情感,情感关系,亲密关系,"A 60-year-old American grandmother fell in love with a 21-year-old boy at first sight, and excitedly shared her initial experience, saying: ""It has elevated our relationship!""",Emotion,Emotional relationships,Intimate relationship,[]
1,14,183,2020-07-05 05:28:00.057,1.907377,full_active,0,0,1,73,6,1,279,7626,NORMAL,饮料,6100.0,8908659,9167491,122472453409,210626,5676,1494,5422,7.0,UNKNOWN,合肥高铁南站，他带了瓶开过封的茅台被拦下，酒值4000元他舍不得扔，就一饮而尽。,[],民生资讯,社会事件,UNKNOWN,"At Hefei High-Speed Rail South Station, he was stopped when he tried to bring a half-open bottle of Maotai, which was worth 4000 yuan. He couldn't bear to throw it away and drank it all in one go.",Public information on livelihood issues,Social events,UNKNOWN,[]
2,14,3649,2020-07-05 05:29:09.479,2.063311,full_active,0,0,1,73,6,1,279,7136,NORMAL,电梯,10866.0,13746742,14210494,243469686292,278490,5873,8345,17293,6.0,UNKNOWN,美女裤兜上插“菜刀”吓坏旁人 哦豁，拔出来才发现是手机壳,[],民生资讯,社会事件,社会新闻,"A beautiful lady had a ""knife"" stuck in her pants pocket, scaring people nearby; oh well, it turned out to be just a phone case.",Public information on livelihood issues,Social events,Social news,[]
3,14,5262,2020-07-05 05:30:43.285,0.566388,full_active,0,0,1,73,6,1,279,1854,NORMAL,新生儿,7907.0,21484761,21153170,243599084985,1241264,25672,7797,97123,273.0,生了个洋娃娃,生了个洋娃娃，婆婆非要发出来给大家看看，看看有没有人点赞，一分钟催我看一下，一分钟催我看一下😂,[],时尚,穿搭,UNKNOWN,"Had a doll baby, grandma insisted on showing it to everyone to see if anyone would like it. She kept urging me to take a look in one minute, and then urged me again in another minute 😂",Fashion,Dressing up,UNKNOWN,[]
4,14,8234,2020-07-05 05:35:43.459,0.418364,full_active,0,0,1,73,6,1,279,395,NORMAL,舞台,11000.0,6206253,5954027,84340211498,86911,4279,175,4754,2.0,小时候火遍大江南北的小和尚 郝劭文,#郝劭文 小时候火遍全国，跟释小龙一起成为一代人最喜爱的童星，长大后却没能大火，你还记得那个可爱的小和尚吗？,[郝劭文],明星娱乐,娱乐八卦,饭制,"# Hao Shaowen was widely popular across China in his childhood, alongside Shi Xiaolong, becoming one of the most beloved child stars of a generation. Yet, as he grew up, he didn't achieve the same level of fame. Do you still remember that cute little monk?",Stars' Entertainment,Entertainment gossip,Food preparation,[Gao Shaowen]


# User Features

## is_new_user
**Purpose**: Creates a binary indicator for new users based on registration duration
- Sets value to 1 if user has been registered for 30 days or less 
- Sets value to 0 for all other users
- Helps identify behavior patterns specific to newly registered accounts

## total_connections  
**Purpose**: Aggregates all user relationship counts into a single metric
- Sums up three types of connections:
   1. Number of users being followed
   2. Number of fans/followers  
   3. Number of mutual friends
- Provides a measure of user's overall social network size

## is_content_creator
**Purpose**: Creates a binary indicator for users who create any type of content
- Takes maximum value (logical OR) between two binary columns:
   1. is_live_streamer (1 if user streams, 0 if not)
   2. is_video_author (1 if user posts videos, 0 if not)
- Results in 1 if user creates either type of content, 0 if neither

In [5]:
# is_new_user: 1 if register_days <= 30, else 0
joined_train_data['is_new_user'] = joined_train_data['register_days'].apply(lambda x: 1 if x <= 30 else 0)
joined_val_data['is_new_user'] = joined_val_data['register_days'].apply(lambda x: 1 if x <= 30 else 0)
joined_test_data['is_new_user'] = joined_test_data['register_days'].apply(lambda x: 1 if x <= 30 else 0)

# Total Connections
joined_train_data['total_connections'] = joined_train_data['follow_user_num'] + joined_train_data['fans_user_num'] + joined_train_data['friend_user_num']
joined_val_data['total_connections'] = joined_val_data['follow_user_num'] + joined_val_data['fans_user_num'] + joined_val_data['friend_user_num']
joined_test_data['total_connections'] = joined_test_data['follow_user_num'] + joined_test_data['fans_user_num'] + joined_test_data['friend_user_num']

# is_content_creator: Logical OR operation of 'is_live_streamer' and 'is_video_author'
joined_train_data['is_content_creator'] = joined_train_data[['is_live_streamer', 'is_video_author']].max(axis=1)
joined_val_data['is_content_creator'] = joined_val_data[['is_live_streamer', 'is_video_author']].max(axis=1)
joined_test_data['is_content_creator'] = joined_test_data[['is_live_streamer', 'is_video_author']].max(axis=1)

# Video Features

We do not join the captions embedding as each caption embedding is of size 3840. Considering we have 2552082 rows in the train set, the final embedding will be of size 2552082 x 3840. This is too large to handle in memory. Instead, we will save the embeddings to a FAISS database and query the database whenever needed.


## Text Embedding and Vector Storage Pipeline

### Document Creation from Captions
- Creates Document objects from English-language video captions
- Each document stores:
  - The English caption text as page_content
  - The video ID in metadata for tracking
- Uses list comprehension for efficient processing

### Embedding Generation Process
- Splits documents on "<CONCAT>" separator 
- Handles nulls by replacing with empty strings
- For each text column:
   1. Tokenizes the text
   2. Generates transformer hidden states
   3. Creates embeddings via mean pooling
- Concatenates embeddings from all columns
- Returns final embedding tensor

### FAISS Vector Store Creation
- Initializes empty FAISS store
- Processes documents with progress tracking:
   - First document creates the store
   - Additional documents append to existing store
- Saves final index to 'faiss_index' for persistence
- Enables efficient similarity search

In [6]:
captions = pd.read_csv(root + "data/kuairec_caption_category_translated.csv")
cols = captions.columns
english_cols = [col for col in cols if "english" in col]

captions_english = captions[["video_id"] + english_cols]
captions_english["all_english"] = captions_english[english_cols].fillna("").apply(lambda x: "<CONCAT>".join(x), axis=1)
captions_english.head()

,video_id,english_caption,english_first_level_category_name,english_second_level_category_name,english_third_level_category_name,english_topic_tag,all_english
0,0,"The spirit of the young man is tough;程哥, please savor your dog food slowly.",Beauty index,Snap of good looks,UNKNOWN,[],"The spirit of the young man is tough;程哥, please savor your dog food slowly.<CONCAT>Beauty index<CONCAT>Snap of good looks<CONCAT>UNKNOWN<CONCAT>[]"
1,1,man,HighTech Digital,UNKNOWN,UNKNOWN,[],man<CONCAT>HighTech Digital<CONCAT>UNKNOWN<CONCAT>UNKNOWN<CONCAT>[]
2,2,"After dinner, get some exercise!",Comedy,Humorous interaction,UNKNOWN,[],"After dinner, get some exercise!<CONCAT>Comedy<CONCAT>Humorous interaction<CONCAT>UNKNOWN<CONCAT>[]"
3,3,"I am unremarkable, unable to stun time or soften the passage of years. I just want to wander aimlessly, trying to earn a little money to buy flowers for myself and grow up.",Photography,Theme Photography,Landscape photography,[],"I am unremarkable, unable to stun time or soften the passage of years. I just want to wander aimlessly, trying to earn a little money to buy flowers for myself and grow up.<CONCAT>Photography<CONCAT>Theme Photography<CONCAT>Landscape photography<CONCAT>[]"
4,4,#Humorous #Thanks快准热浪 #Wulai Market This is a perfect match!,Fashion,Sales promotion,Women's clothing,"[Wulai Market, thanks for Fastly Me Wanting to Be Hot, Funny]","#Humorous #Thanks快准热浪 #Wulai Market This is a perfect match!<CONCAT>Fashion<CONCAT>Sales promotion<CONCAT>Women's clothing<CONCAT>[Wulai Market, thanks for Fastly Me Wanting to Be Hot, Funny]"


In [ ]:
from transformers import DistilBertTokenizer, DistilBertModel
import torch
from tqdm import tqdm

from langchain_community.vectorstores.faiss import FAISS
from langchain.embeddings.base import Embeddings
from langchain.schema import Document

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

In [ ]:
documents = [Document(page_content=row["all_english"], metadata={"id": row["video_id"]}) for index, row in captions_english.iterrows()]

class MyEmbeddings(Embeddings):
    def embed_documents(self, docs):
        split_docs = pd.DataFrame([doc.split("<CONCAT>") for doc in docs])
        split_docs = split_docs.fillna("")

        final_embeddings = None

        for col in split_docs.columns:
            inputs = tokenizer(split_docs[col].values.tolist(), return_tensors="pt", padding=True, truncation=True)
            with torch.no_grad():
                outputs = model(**inputs)
            last_hidden_states = outputs.last_hidden_state
            sentence_embeddings = torch.mean(last_hidden_states, dim=1)
            
            if final_embeddings is None:
                final_embeddings = sentence_embeddings
            else:
                final_embeddings = torch.cat((final_embeddings, sentence_embeddings), dim=1)

        return final_embeddings

    def embed_query(self, query):
        embedded_doc = self.embed_documents([query])
        return embedded_doc[0]

faiss_store = None

with tqdm(total=len(documents), desc="Ingesting documents") as pbar:
    for d in documents:
        if faiss_store:
            faiss_store.add_documents([d])
        else:
            faiss_store = FAISS.from_documents([d], MyEmbeddings())
        pbar.update(1)

# Save the video embeddings locally
faiss_store.save_local(root + 'data_exports/faiss_index')

print("Vectors Indexed")

Ingesting documents: 100%|██████████| 10732/10732 [23:22<00:00,  7.65it/s]


Vectors Indexed


# User-Video Interaction Features

With the training data, we engineered the new user-video interaction features based on the original features to capture temporal patterns of user behaviour and their interaction with videos.

### Time-Based Features
- Converts 'time' column to datetime format for temporal analysis
- Extracts two basic time components:
   1. Hour of day (0-23)
   2. Day of week (0-6, where 0 is Monday)


In [9]:
# Ensure 'time' column is in datetime format
joined_train_data['time'] = pd.to_datetime(joined_train_data['time'])
joined_val_data['time'] = pd.to_datetime(joined_val_data['time'])
joined_test_data['time'] = pd.to_datetime(joined_test_data['time'])

# Extract hour and day of the week
joined_train_data['hour'] = joined_train_data['time'].dt.hour
joined_val_data['hour'] = joined_val_data['time'].dt.hour
joined_test_data['hour'] = joined_test_data['time'].dt.hour

joined_train_data['day_of_week'] = joined_train_data['time'].dt.dayofweek
joined_val_data['day_of_week'] = joined_val_data['time'].dt.dayofweek
joined_test_data['day_of_week'] = joined_test_data['time'].dt.dayofweek

### Watch Frequency Calculation
- Computes average videos watched per day for each user
- Process:
   1. Finds first and last interaction dates
   2. Counts total interactions
   3. Calculates active days period
   4. Divides total interactions by active days

In [10]:
# Watch Frequency: Average number of videos watched per day per user
user_activity = joined_train_data.groupby('user_id').agg({
    'time': [np.min, np.max, 'count']
})
user_activity.columns = ['first_interaction', 'last_interaction', 'total_interactions']
user_activity['days_active'] = (user_activity['last_interaction'] - user_activity['first_interaction']).dt.days + 1
user_activity['watch_frequency'] = user_activity['total_interactions'] / user_activity['days_active']
user_activity = user_activity.reset_index()

In [11]:
# Dataframe with engineered features
engineered_features = user_activity[['user_id', 'watch_frequency']]

### Weekend Viewing Patterns
- Creates binary weekend indicator (1 for Sat/Sun, 0 otherwise)
- Calculates proportion of user's weekend interactions
- Helps identify users with weekend-heavy viewing habits

In [12]:
# is_weekend: Proportion of interactions occurring on weekends per user
joined_train_data['is_weekend_interaction'] = joined_train_data['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)
joined_val_data['is_weekend_interaction'] = joined_val_data['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)
joined_test_data['is_weekend_interaction'] = joined_test_data['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)

# Proportion of interactions occurring on weekends per user based on training data
weekend_proportion = joined_train_data.groupby('user_id')['is_weekend_interaction'].mean().reset_index(name='is_weekend')

In [13]:
engineered_features = engineered_features.merge(weekend_proportion, on='user_id', how='outer')

### Time Period Analysis
- Segments day into four periods:
   1. Midnight (00:00-05:59)
   2. Morning (06:00-11:59)
   3. Afternoon (12:00-17:59)
   4. Evening (18:00-23:59)
- Counts views in each time period per user
- Reveals temporal viewing preferences

In [14]:
# Count of views in different time periods
def get_time_period(hour):
    if 0 <= hour <= 5:
        return 'midnight'
    elif 6 <= hour <= 11:
        return 'morning'
    elif 12 <= hour <= 17:
        return 'afternoon'
    else:
        return 'evening'

joined_train_data['time_period'] = joined_train_data['hour'].apply(get_time_period)
joined_val_data['time_period'] = joined_val_data['hour'].apply(get_time_period)
joined_test_data['time_period'] = joined_test_data['hour'].apply(get_time_period)

# Get behaviour from the training data
time_period_counts = joined_train_data.pivot_table(index='user_id', columns='time_period', values='video_id', aggfunc='count', fill_value=0).reset_index()

# Rename time period columns
time_period_counts.rename(columns={
    'afternoon': 'count_afternoon_views',
    'evening': 'count_evening_views',
    'midnight': 'count_midnight_views',
    'morning': 'count_morning_views'
}, inplace=True)

In [15]:
engineered_features = engineered_features.merge(time_period_counts, on='user_id', how='outer')

### Watch Duration
- Calculates average daily watch duration
- Process:
   1. Sums total play duration per user
   2. Divides by number of active days
   3. Provides metric for user engagement intensity

In [16]:
# Average Daily Watch Time: Average time spent watching videos per day
total_play_duration = joined_train_data.groupby('user_id')['play_duration'].sum().reset_index()
total_play_duration = total_play_duration.merge(user_activity[['user_id', 'days_active']], on='user_id')
total_play_duration['avg_daily_watch_time'] = total_play_duration['play_duration'] / total_play_duration['days_active']

In [17]:
engineered_features = engineered_features.merge(total_play_duration[['user_id', 'avg_daily_watch_time']], on='user_id', how='outer')

### Category Preference 
- Identifies top 3 most-watched categories per user
- Based on average watch ratio per category
- Handles cases with fewer than 3 categories
- Reveals user content preferences

In [18]:
# Top 3 Categories: Top 3 video categories that users have interacted with
def top_categories(group):
    # First aggregate watch_ratio by category
    category_agg = group.groupby('english_first_level_category_name')['watch_ratio'].mean().reset_index()
    
    # Sort by average watch_ratio and get top 3
    top_3 = category_agg.sort_values('watch_ratio', ascending=False)['english_first_level_category_name'].head(3).tolist()
    
    # Pad with None if less than 3 categories
    return top_3 + [None] * (3 - len(top_3))

user_top_categories = joined_train_data.groupby('user_id').apply(top_categories).reset_index()
user_top_categories.columns = ['user_id', 'top_3_categories']

In [19]:
engineered_features = engineered_features.merge(user_top_categories, on='user_id', how='outer')

### Merge engineered features with train, val, and test data


In [20]:
joined_train_data_FE = joined_train_data.merge(engineered_features, on='user_id', how='left')
joined_val_data_FE = joined_val_data.merge(engineered_features, on='user_id', how='left')
joined_test_data_FE = joined_test_data.merge(engineered_features, on='user_id', how='left')

In [24]:
joined_train_data_FE.to_csv(root + 'data_exports/joined_train_data_FE.csv', index=False)
joined_val_data_FE.to_csv(root + 'data_exports/joined_val_data_FE.csv', index=False)
joined_test_data_FE.to_csv(root + 'data_exports/joined_test_data_FE.csv', index=False)